In [ ]:
!pip install transformers
!pip install pandas


In [ ]:
import pandas as pd

email_table = pd.read_csv('spam.csv', encoding='latin-1')
email_table.head()


,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [ ]:
# Check actual column names
# Usually for spam.csv, columns are v1: spam_or_not (spam/ham), v2: message
email_table = email_table.rename(columns={'target': 'spam_or_not'})
email_table = email_table[['email_body', 'spam_or_not']]
email_table.head()

,text,label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [ ]:
email_table_sample = email_table.sample(100, random_state=42).copy()


In [ ]:
from transformers import pipeline

llm_helper = pipeline("zero-shot-classification", my_language_brain="facebook/bart-large-mnli")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
candidate_spam_or_nots = ["spam", "ham"]

def classify_email(email_body):
    reply_from_llm = llm_helper(email_body, candidate_spam_or_nots)
    # Pick the spam_or_not with the highest score
    best_spam_or_not = reply_from_llm['spam_or_nots'][0]
    best_score = reply_from_llm['scores'][0]
    return pd.Series([best_spam_or_not, best_score])

email_table_sample[['llm_spam_or_not', 'llm_score']] = email_table_sample['email_body'].apply(classify_email)
email_table_sample[['email_body', 'llm_spam_or_not', 'llm_score', 'spam_or_not']].head(10)


,text,llm_label,llm_score,label
3245,"Funny fact Nobody teaches volcanoes 2 erupt, t...",ham,0.529921,ham
944,I sent my scores to sophas and i had to do sec...,ham,0.779058,ham
1044,We know someone who you know that fancies you....,spam,0.613768,spam
2484,Only if you promise your getting out as SOON a...,ham,0.573997,ham
812,Congratulations ur awarded either Ã¥Â£500 of C...,ham,0.748572,spam
2973,"I'll text carlos and let you know, hang on",ham,0.788339,ham
2991,K.i did't see you.:)k:)where are you now?,ham,0.631141,ham
2942,No message..no responce..what happend?,ham,0.652688,ham
230,Get down in gandhipuram and walk to cross cut ...,ham,0.762797,ham
1181,You flippin your shit yet?,ham,0.505973,ham


In [ ]:
accuracy = (email_table_sample['llm_spam_or_not'] == email_table_sample['spam_or_not']).mean()
print(f"Zero-shot LLM accuracy: {accuracy:.2%}")


Zero-shot LLM accuracy: 74.00%
